In [9]:
import pandas as pd
import re

In [10]:
# Cargar el archivo CSV
dfdata = pd.read_csv('../Databases/games_cleaned.csv')

In [ ]:
# Mostrar el DataFrame original
print("DataFrame original:")
# print(dfdata.head())
print(dfdata.shape)

DataFrame original:


(111452, 39)

Al tener los datos, vamos a usar Supported languages, categorias, genres y tags como variables categoricas.

In [12]:
def clean_language(lang_str):
    if pd.isna(lang_str):
        return []
    
    # Eliminar corchetes, comillas y caracteres innecesarios
    lang_str = re.sub(r"[\[\]']", '', str(lang_str))
    
    # Dividir los idiomas si hay varios
    if '\\r\\n' in lang_str:
        languages = lang_str.split('\\r\\n')
    else:
        languages = lang_str.split(',')
    
    cleaned_langs = []
    for lang in languages:
        # Limpiar cada idioma
        lang = re.sub(r'&amp;lt;.*?&amp;gt;', '', lang)  # Eliminar HTML entities
        lang = re.sub(r'\\r\\n|\\b', '', lang)  # Eliminar escape sequences
        lang = re.sub(r'\s*\([^)]*\)', '', lang)  # Eliminar texto entre paréntesis
        lang = re.sub(r'b/b$', '', lang)  # Eliminar sufijo b/b
        lang = re.sub(r'[;#_]', '', lang)  # Eliminar caracteres especiales
        lang = re.sub(r'\s+', ' ', lang)  # Reemplazar múltiples espacios por uno solo
        
        # Estandarizar nombres de idiomas
        lang = re.sub(r'Spanish\s*-\s*(Spain|Latin America)', 'Spanish', lang)
        lang = re.sub(r'Portuguese\s*-\s*(Brazil|Portugal)', 'Portuguese', lang)
        lang = re.sub(r'#lang_français|Langfrançais', 'French', lang)
        lang = re.sub(r'English Dutch  English', 'English', lang)  # Corregir duplicados
        lang = re.sub(r'English,', 'English', lang)  # Corregir errores de coma
        lang = re.sub(r'French,', 'French', lang) 
        lang = re.sub(r'Langfrançais', 'French', lang) 
        
        
        # Limpiar errores específicos
        lang = re.sub(r'Englishb/B', 'English', lang)  # Corregir 'Englishb/B'
        lang = re.sub(r'Frenchb/B', 'French', lang)  # Corregir 'Frenchb/B'
        lang = re.sub(r'Germanb/B', 'German', lang)  # Corregir 'Germanb/B'
        lang = re.sub(r'Italianb/B', 'Italian', lang)  # Corregir 'Italianb/B'
        lang = re.sub(r'Koreanb/B', 'Korean', lang)  # Corregir 'Koreanb/B'
        lang = re.sub(r'English Dutch English', 'English', lang)        
        
        # Eliminar espacios extras y convertir a título
        lang = lang.strip().title()
        
        
        if lang and lang.lower() != 'b/b':  # Solo agregar si no está vacío o es un error
            cleaned_langs.append(lang)

    
    
    return cleaned_langs


Aqui se estan quitando a mano estos datos ya que por alguna razon el codigo os estaba generando y no se encontraba forma de borrarlos en otro lado

In [13]:
# # Aplicar la limpieza a la columna de idiomas
all_languages = []
for lang_list in dfdata['Supported languages']:
    all_languages.extend(clean_language(lang_list))

# Obtener valores únicos y ordenarlos
unique_languages = sorted(set(all_languages))
unique_languages.remove('Langfrançais')
unique_languages.remove('Englishb/B, French Germanb/B, Italian, Spanish')
unique_languages.remove('Englishb/B, Frenchb/B, Italianb/B, Germanb/B, Spanishb/B, Russian, Polish, Koreanb/B, Japanese')
unique_languages.remove('Englishb/B, German, French Italian, Spanish, Russian')
unique_languages.remove('Englishb/B, German, French Polish, Spanish, Italian')



In [14]:
print("Lista de idiomas únicos después de la limpieza:")
print(unique_languages)
print(f"\nNúmero total de idiomas únicos: {len(unique_languages)}")


Lista de idiomas únicos después de la limpieza:
['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Assamese', 'Azerbaijani', 'Bangla', 'Basque', 'Belarusian', 'Bosnian', 'Bulgarian', 'Catalan', 'Cherokee', 'Croatian', 'Czech', 'Danish', 'Dari', 'Dutch', 'English', 'Estonian', 'Filipino', 'Finnish', 'French', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Hausa', 'Hebrew', 'Hindi', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Kannada', 'Kazakh', 'Khmer', 'Kiche', 'Kinyarwanda', 'Konkani', 'Korean', 'Kyrgyz', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 'Nepali', 'Norwegian', 'Odia', 'Persian', 'Polish', 'Portuguese', 'Punjabi', 'Quechua', 'Romanian', 'Russian', 'Scots', 'Serbian', 'Simplified Chinese', 'Sindhi', 'Sinhala', 'Slovak', 'Slovakian', 'Slovenian', 'Sorani', 'Sotho', 'Spanish', 'Swahili', 'Swedish', 'Tajik', 'Tamil', 'Tatar', 'Telugu', 'Thai', 'Tig

In [15]:
dflanguages = dfdata.copy()

In [ ]:
for idioma in unique_languages:
    # Crear el nombre de la columna (en minúsculas y reemplazando espacios por guiones bajos)
    nombre_columna = f'language_{idioma.lower().replace(" ", "_")}'
    
    # Crear la columna binaria
    # Será 1 si el idioma está en la lista de idiomas soportados del juego, 0 si no
    dflanguages[nombre_columna] = dflanguages['Supported languages'].apply(lambda x: 1 if idioma in x else 0)

# Mostrar las primeras filas del DataFrame con las nuevas columnas
# print(dflanguages.head())


(111452, 140)


In [24]:
print("Database modificada: ")
print(dflanguages.shape)

Database modificada: 
(111452, 140)
